In [ ]:
/**
 * @file rocket-injector-design-cpp.ipynb
 * 
 * This notebook demonstrates a practical application of Multi-Objective Optimizers. The design of the injector surface is
 * modified to study its effect on the thrust chamber environment and performance. The optimizer simultaneously balances these   
 * objectives to returns a set of Pareto optimal solutions.
 *
 * Also known as RE3-4-7, this problem has been taken from "An Easy-to-use Real-world Multi-objective
 * Optimization Problem Suite" paper. For more information, visit: https://github.com/ryojitanabe/reproblems.
 */

In [ ]:
#include <mlpack/xeus-cling.hpp>

#include <ensmallen.hpp>

In [ ]:
// Header files to create and show the plot.
#define WITHOUT_NUMPY 1
#include "matplotlibcpp.h"
#include "xwidgets/ximage.hpp"

namespace plt = matplotlibcpp;

In [ ]:
using namespace ens;

In [ ]:
using namespace ens::test;

### 1. Background

It was the June of 1962, the company Rocketdyne was tasked with the heavy responsibility of sending a man to the moon as per Kennedy's bold promise. NASA, along with Rocketdyne, developed an F-1 engine capable of burning fuel of Olympic size swimming pool. On the D-day, when the engines ignited and the rocket was ready to take off, the engines exploded and the failure was catastrophic.

**So, what went wrong?**

Let's discuss the basics first.

A rocket system consists of two liquid propellants, namely the fuel ($H_2$) and oxidiser ($O_2$). These propellants are stored in separate tanks, and pumped into the combustion chamber. The chamber houses the exothermic reaction between the propellants to release energy vital for the take-off process. For the reaction to occur readily, an injector mechanism is installed, which disperses these liquid propellants into tiny droplets and allow the mixing of the chemicals.

![](media/combustion.gif)

 **The Achilles heel**
 
A good injector design makes all the difference between your system performing excellently and it exploding. Unfortunately, the latter was the case for the ambitious project. 

Why so? The devil's in the details. The combustion process creates extreme temperature and pressure conditions inside the thrust chamber. Under such circumstances, the chamber is a very sensible high-pressure cooking pot. Thus, if the influx is off ever so slightly the flames would sway controllably burning down the entire system. 

 ![](media/unstable.gif)

### 2. Framing the problem.

With the motivation established, let's begin by framing the problem. As discussed, the cornerstone of an ideal rocket system is its injector design. One can infer that the performance and quality of life of the system stand in stark contrast. Both of these are primarily controlled by injector design. This is where multi-objective optimizers kick in. In this notebook, we will try and find the optimal set of design variables to generate a set of [Pareto Optimal](https://www.investopedia.com/terms/p/pareto-efficiency.asp) solutions.


#### I. Design Variables 

The values are normalized to the range [0, 1].

a) **Hydrogen Flow Angle** ($\alpha$) : The acute angle formed between the fuel and oxidiser. 

b) **Hydrogen Area** ($H.A$):  The area of the tube from which $H_2$ flows in.

c) **Oxygen Area** ($O.A$): The area of the tube from which $O_2$ flows in.

d) **Oxygen Post-Tip-Thickness** ($O.P.T.T$): The thickness of the annulus between the two propellants.

![](media/design.jpg)

#### II. Objective Functions

All the objectives below should be **minimized**. Further, each of the following output have been normalized to [0, 1] range.

a) **Maximum temperature at injector face** ($TF_{max}$): Prevent meltdown of the injector surface. Determines the thermal stability of the combustion chamber. 

b) **Distance From inlet** ($X_{CC}$): The distance from the injector surface where 90% of the combustion is complete. Lesser   values signifies quality mixing and high performance.

c) **Maximum temperature at post tip** ($TT_{max}$): The temperature at the annulus of the propellants tubes. Similar to $TF_{max}$, this objective is to be minimized to prevent thermal breakdown.

![](media/objectives.jpg)   

                                                

In [ ]:
class InjectorDesignProblem
{
  public:
    InjectorDesignProblem()
    { /* Nothing to do here. */ }
    
    //! Get the starting point.
    arma::mat GetInitialPoint()
    {
      return arma::mat(numVariables, 1, arma::fill::zeros);
    }
    
    //! The maximum temperature at the injector face.
    struct InjectorFaceHeatCap
    {
        InjectorFaceHeatCap() {}

        double Evaluate(const arma::mat& coords)
        {
            DesignVariables x(coords);
            return 0.692 + (0.477 * x.Alpha) - (0.687 * x.HA) - (0.080 * x.OA) - (0.0650 * x.OPTT) - (0.167 * x.Alpha * x.Alpha)
                - (0.0129 * x.HA * x.Alpha) + (0.0796 * x.HA * x.HA) - (0.0634 * x.OA * x.Alpha) - (0.0257 * x.OA * x.HA)
                + (0.0877 * x.OA * x.OA) - (0.0521 * x.OPTT * x.Alpha) + (0.00156 * x.OPTT * x.HA) + (0.00198 * x.OPTT * x.OA)
                + (0.0184 * x.OPTT * x.OPTT);
        }    
    };
    
    //! Distance from inlet where 90% combustion is complete.
    struct DistanceFromInlet
    {
        DistanceFromInlet() {}
        
        double Evaluate(const arma::mat& coords)
        {
            DesignVariables x(coords);
            return 0.153 - (0.322 * x.Alpha) + (0.396 * x.HA) + (0.424 * x.OA) + (0.0226 * x.OPTT)
                + (0.175 * x.Alpha * x.Alpha) + (0.0185 * x.HA * x.Alpha) - (0.0701 * x.HA * x.HA) - (0.251 * x.OA * x.Alpha)
                + (0.179 * x.OA * x.HA) + (0.0150 * x.OA * x.OA) + (0.0134 * x.OPTT * x.Alpha) + (0.0296 * x.OPTT * x.HA)
                + (0.0752 * x.OPTT * x.OA) + (0.0192 * x.OPTT * x.OPTT);
        }
    };
    
    //! The maximum temperature at the post tip.
    struct PostTipHeatCap
    {
        PostTipHeatCap() {}
        
        double Evaluate(const arma::mat& coords)
        {
            DesignVariables x(coords);
            return 0.370 - (0.205 * x.Alpha) + (0.0307 * x.HA) + (0.108 * x.OA) + (1.019 * x.OPTT)
                - (0.135 * x.Alpha * x.Alpha) + (0.0141 * x.HA * x.Alpha) + (0.0998 * x.HA * x.HA) + (0.208 * x.OA * x.Alpha)
                - (0.0301 * x.OA * x.HA) - (0.226 * x.OA * x.OA) + (0.353 * x.OPTT * x.Alpha) - (0.0497 * x.OPTT * x.OA)
                - (0.423 * x.OPTT * x.OPTT) + (0.202 * x.HA * x.Alpha * x.Alpha) - (0.281 * x.OA * x.Alpha * x.Alpha)
                - (0.342 * x.HA * x.HA * x.Alpha) - (0.245 * x.HA * x.HA * x.OA) + (0.281 * x.OA * x.OA * x.HA)
                - (0.184 * x.OPTT * x.OPTT * x.Alpha) - (0.281 * x.HA * x.Alpha * x.OA);
        }

    };

    //! Get objective functions.
    std::tuple<InjectorFaceHeatCap, DistanceFromInlet, PostTipHeatCap> GetObjectives()
    {
      return std::make_tuple(InjectorFaceHeatCap{}, DistanceFromInlet{}, PostTipHeatCap{});
    }
                        
  private:
    //! A wrapper for the design variables. Used for better
    //! readibility.
    struct DesignVariables
    {
        DesignVariables(const arma::mat& coords)
        {
            Alpha = coords[0];
            HA = coords[1];
            OA = coords[2];
            OPTT = coords[3];
        }
        
        //! Hydrogen flow angle.
        double Alpha;
        //! Hydrogen area.
        double HA;
        //! Oxygen area.
        double OA;
        //! Oxidiser Post Tip Thickness.
        double OPTT;
    };
    
    size_t numVariables = 4;
    size_t numObjectives = 3;
};

InjectorDesignProblem idp;
auto objectives = idp.GetObjectives();

In [ ]:
const double lowerBound = 0;
const double upperBound = 1;

DefaultMOEAD moead(300, // Population size.
                   150,  // Max generations.
                   1.0,  // Crossover probability.
                   0.9, // Probability of sampling from neighbor.
                   20, // Neighborhood size.
                   20, // Perturbation index.
                   0.5, // Differential weight.
                   2, // Max childrens to replace parents.
                   1E-10, // epsilon.
                   lowerBound, // Lower bound.
                   upperBound // Upper bound.
                 );

arma::mat coords = idp.GetInitialPoint();

In [ ]:
moead.Optimize(objectives, coords);

Store the optimal set of solutions in a ```arma::cube``` data structure. Each slice of the cube is a ```3 x 1``` matrix 
representing a single set of optimal solution.

In [ ]:
const arma::cube& paretoFront = moead.ParetoFront();

In [ ]:
size_t populationSize = paretoFront.n_slices;

std::vector<double> X (populationSize, 0.), 
                    Y (populationSize, 0.), 
                    Z (populationSize, 0.);

for (size_t idx = 0; idx < populationSize; ++idx)
{
    X[idx] = paretoFront.slice(idx)(0);
    Y[idx] = paretoFront.slice(idx)(1);
    Z[idx] = paretoFront.slice(idx)(2);
}

In [ ]:
plt::figure_size(1000, 800);
plt::suptitle("Scatter Matrix of Pareto Fronts");
plt::subplot(3, 1, 1);
plt::scatter(X, Y, 50);
plt::xlabel("InjectorFaceHeatCap");
plt::ylabel("DistanceFromInlet");

plt::subplot(3, 1, 2);
plt::scatter(X, Z, 50);
plt::xlabel("InjectorFaceHeatCap");
plt::ylabel("PostTipHeatCap");

plt::subplot(3, 1, 3);
plt::scatter(Y, Z, 50);
plt::xlabel("DistanceFromInlet");
plt::ylabel("PostTipHeatCap");

plt::save("./scatter.png");
auto im = xw::image_from_file("scatter.png").finalize();
im

### References

1. "NASA's Baffling Engine Problem", Youtube, uploaded by Primal Space, 13th March 2020, https://www.youtube.com/watch?v=xbvQBwnppQo.
2. "LIQUID PROPELLANT ROCKET ENGINE/liquid rocket 3d animation/construction working/ LEARN FROM THE BASE", Youtube, uploaded by Learn from the base. 12th September 2020, https://www.youtube.com/watch?v=9Y3fG-YrIII.
3. "Rocket Fuel Injectors - Things Kerbal Space Program Doesn't Teach", Youtube, uploaded by Scott Manley, 10th March 2020, https://www.youtube.com/watch?v=aa4ATJGRqA0.
4. Tanabe, Ryoji, and Hisao Ishibuchi. "An easy-to-use real-world multi-objective optimization problem suite." Applied Soft Computing 89 (2020): 106078.
5. Vaidyanathan, Rajkumar, et al. "Cfd-based design optimization for single element rocket injector." 41st Aerospace Sciences Meeting and Exhibit. 2003.